# Heuristic V1

In [1]:
import pickle
import numpy as np
import StrokeDef
import xmlparse
import xmltodict
import score_strokes

from StrokeDef import loadStrokeDict
from xmlparse import loadGeometryBases, loadRef
from score_strokes import strokeError, strokeErrorScaled

"""

"""

'\n\n'

## Comparing Gene Strokes to Archetypal Strokes

In [9]:
archetypal_strokes = loadStrokeDict()
num = 0
for key in archetypal_strokes:
    print(key)
    print(num)
    #print(archetypal_strokes[key].arial)
    print()
    num=num+1
    

"""
Given a gene's list of strokes and percentage progresses, gives the likely number of each archetypal stroke
contained in the gene


Parameters:
    g_stroke_list - list of strokes contained for a given gene file
    
    g_percent_list - list of stroke percentage progresses (corresponding to g_stroke_list) for a given gene file

Return Value:
    total_array - an array of 55 elements, containing the total (summed) percentages for each stroke type in the entire gene
    
    percent_arrays - a 2D array with the percentages for each individual stroke (dimensions are number of strokes x 55)
"""
def getGeneStrokePercentages(g_stroke_list, g_percent_list):
    total_array = np.zeros(55)
    percent_arrays = []
    stroke_count = 0
    print(len(g_stroke_list))
    print("_____")
    for g_points, g_frac in zip(g_stroke_list, g_percent_list):
        percent_arrays.append(getStrokePercentage(g_points, g_frac))
        #"""
        total_array = total_array+percent_arrays[stroke_count]
        stroke_count = stroke_count+1
        #"""
        print()
    return total_array, percent_arrays


"""
Gives a list of scores for how well a given gene stroke matches each of the archetypal strokes

Parameters:
    g_points - a list of the points in the gene stroke
    g_frac_prog - a list of fractional progress of each point in the gene stroke
    
Return Value:
    strokePercentages - an array containing 55 elements, each representing how well geneStroke
    compares to each of the archetypal strokes
"""
def getStrokePercentage(g_points, g_frac_prog):
    
    strokePercentages = []
    
    [xMin, yMin] = g_points[0]
    xMax = 0
    yMax = 0
    for [x, y] in g_points:
        if (x < xMin):
            xMin = x
        elif (x > xMax):
            xMax = x
        if (y < yMin):
            yMin = y
        elif (y > yMax):
            yMax = y
    
    for key in archetypal_strokes:
        print(key)
        stroke_arial = removeDoubles(archetypal_strokes[key].arial)
        
        print(g_points)
        print()
        """
        
        print(archetypal_strokes[key].arial)
        print()
        print(stroke_arial)
        print()
        """
        points = convertToPointList(stroke_arial, xMin, xMax, yMin, yMax)
        print(key)
        print("points")
        print(points)
        frac_prog = getFractionalProgress(points)
        print("frac_prog")
        print(frac_prog)
        
        """
        print()
        print("~~~~~~~~~~")
        print()
        """
        error = strokeErrorScaled(g_points, points, g_frac_prog, frac_prog)
        #error = error*error # to increase difference between small and large errors
        print(error)
        if error == 0:
            error = 1 # converts to perfect score
        else:
            error = 1.0/error # converts to score
            if error > 1:
                error = 1
        strokePercentages.append(error)
    
    """
    highScores = [[0, 0, 0], [0, 0, 0]]
    for i in range(0, len(strokePercentages)):
        j = findSmaller(strokePercentages[i], highScores[0])
        if j != -1:
            print(i)
            print(strokePercentages[i])
            print()
            highScores[0].insert(j, strokePercentages[i])
            highScores[0].pop(3)
            highScores[1].insert(j, i)
            highScores[1].pop(3)
    strokePercentages[:] = [0 for _ in strokePercentages]
    print(highScores[0])
    print(highScores[1])
    for k in range(0, 3):
        strokePercentages[highScores[1][k]] = highScores[0][k]
    """
    strokePercentages = strokePercentages/sum(strokePercentages) # scores now sum to 1
    return strokePercentages    

"""
Finds the first element in a list that is smaller than a given number, returns the index of that element

Parameters:
    num - a given number

Return Value:
    smaller - the index of the first element in a list that is smaller than num
"""
def findSmaller(num, aList):
    smaller = -1
    for i in range(0, len(aList)):
            if num > aList[i] and smaller == -1:
                smaller = i
    return smaller
    

"""
Parameters:
    stroke_arial - arial data from an archetypal stroke to be converted

Return Value:
    adj_arial - arial data without any doubles within the same segment
"""

def removeDoubles(stroke_arial):
    adj_arial = []
    stroke_def = stroke_arial.copy() # to prevent original dictionary from being altered
    for segment in stroke_def:
        new_seg = []
        index = 0
        """
        print("new segment")
        print(segment)
        print()
        """
        for point in segment:
            """
            print("new point")
            print(index)
            print()
            """
            print(point)
            flag = False
            for i in range(index+1, len(segment)):
                if (segment[i] == point).all():
                    flag = True
                    """
                    print("true")
                    print(segment[i]==point)
                    print((segment[i]==point).all())
                    print(i)
                    print(point)
                    print(segment[i])
                    """
            if flag == False:
                new_seg.append(point)
            index = index+1
            """
            print("new index")
            print(index)
            """
        adj_arial.append(new_seg)
        print(new_seg)
    return adj_arial
    
    
"""
Converts the arial data of a StrokeType object (archetypal stroke) to the point list format of a gene stroke
and returns the newly formatted data

Parameters:
    stroke_arial - arial data from an archetypal stroke to be converted
    
Return Value:
    point_list - an array of pairs, stroke_arial converted to the point list format of a gene stroke
"""
def convertToPointList(stroke_arial, xMin, xMax, yMin, yMax):
    point_list = []
    
    # Resizes to match gene stroke
    xDelta = xMax - xMin
     
    yDelta = yMax - yMin
    
    if xDelta == 0 and hasChange(stroke_arial, 0):
        print("X")
        if (yDelta > 1):
            xDelta = yDelta
        else:
            xDelta = 1
    if yDelta == 0 and hasChange(stroke_arial, 1):
        print("Y")
        if (xDelta > 1):
            yDelta = xDelta
        else:
            yDelta = 1 
    for segment in stroke_arial:
        for point in segment:
            #print(point[0])
            point[0] = point[0]*xDelta + xMin
            #print(point[0])
            #print(point[1])
            point[1] = 1 - point[1] # converts from 0 being top to 0 being bottom
            point[1] = point[1]*yDelta + yMin
            #print(point[1])
            
            """
            [newX] = [point[0]]
            [newY]= [point[1]]
            newPoint = []
            newPoint.append(newX)
            newPoint.append(newY)
            point_list.append(newPoint)
            """
            point_list.append(point)
            #print(point)
            #print(point_list)
            #print()
    return point_list

"""
Returns whether or not there is a change of value between a list of points in one dimension

Parameters:
    stroke_arial - the list of points in the arial data format
    dim - the dimension in which distance is being measured (ex. 0 = x, 1 = y)

Return Value:
    boolean representing if change was found or not (True = yes, False = no)
"""
def hasChange(stroke_arial, dim):
    for segment in stroke_arial:
        og = segment[0][dim]
        for i in range(1, len(segment)):
            other = segment[i][dim]
            if og != other:
                return True
    return False
    


"""
Given a list of points in a stroke, returns a list containing the fractional progress of said stroke
that each point corresponds to

Parameters:
    point_list - a list of points for a given stroke
    
Return Value:
    frac_prog - a list containing the fractional progress of a given stroke corresponding to each point
"""
def getFractionalProgress(point_list):
    frac_prog = [0]
    for i in range(len(point_list)-1):
        frac_prog.append(np.linalg.norm((point_list[i]-point_list[i+1]))+frac_prog[-1])
    frac_prog = np.array(frac_prog)
    frac_prog /= frac_prog.max()
    return frac_prog


T1
0

SWZ
1

HZG1
2

SZWG3
3

C0
4

HZT3
5

H1
6

N
7

HZT1
8

XG
9

TN2
10

SZZ2
11

HZWG1
12

SZ2
13

TN1
14

PZ
15

HZG2
16

HZZZ
17

H2
18

ST
19

T2
20

S2
21

HZ
22

WG1
23

SZZ1
24

SWG
25

HZW
26

PG
27

BXG
28

HP1
29

D2
30

HZT2
31

HZWG2
32

C1
33

HPWG
34

D1
35

HP2
36

HZZZG
37

WG3
38

SP
39

P
40

HZZP
41

HG1
42

HZZ
43

C4
44

PD1
45

S1
46

SZWG2
47

D0
48

SG
49

SZWG1
50

PD2
51

WG2
52

HXWG
53

SZ1
54



## Comparing Gene Stroke Percentage to Archetype Stroke Counts

In [3]:
"""
Compares the stroke percentages of a gene to the stroke counts of every archetype in order to
find the top 100 archetypes that most closely resemble the gene

Parameters:
    geneStrokePercents - an array containing the total percentages
    corresponding to the number of archetypal strokes present in the gene
    
    numStrokes - number of strokes contained in the gene

Return Value:
    archetype_matches - a list of the top 50 archetypes whose stroke counts most closely match the gene's stroke percentages

"""
def strokeCountComparison()

SyntaxError: invalid syntax (<ipython-input-3-a928a5df63e4>, line 15)

## Check Archetypes for Highest Score

In [ ]:
"""
IDEAS:
"""

In [ ]:
# test removeDoubles
stroke_name = "HZZZG"
archetypal_strokes = loadStrokeDict()
stroke_arial = removeDoubles(archetypal_strokes[stroke_name].arial)
print(stroke_arial)

In [ ]:
# Test archetypal stroke data
#"""
stroke_name = "HZZZG"
archetypal_strokes = loadStrokeDict()
stroke_arial = archetypal_strokes[stroke_name].arial
print(stroke_arial)
points = convertToPointList(stroke_arial, 0, 1, 0, 1)
print(points)
frac_prog = getFractionalProgress(stroke_arial)
print("frac_prog")
print(frac_prog)
#"""

In [ ]:
# Test getStrokePercentage()
"""
data_dir = "NewGenes"
char_data = loadGeometryBases(data_dir)
g_data, _, base_data, stroke_sets, _, f_names = char_data
p_list = []
for (gl, bases, stroke_set, f_name) in zip(g_data, base_data, stroke_sets, f_names):
        g, l = gl
        print()
        print(l)
        print()
        for g_points, g_frac in zip(g, l):
            print("g_points")
            print(len(g_points))
            print(g_points)
            print()
            print(g_frac)
            p_list.append(getStrokePercentage(g_points, g_frac))
print()
print()
print("Percentages:")
print(p_list)
"""

In [ ]:
# Test getGeneStrokePercentages()
data_dir = "NewGenes"
char_data = loadGeometryBases(data_dir)
g_data, _, base_data, stroke_sets, _, f_names = char_data
p_list = []
for (gl, bases, stroke_set, f_name) in zip(g_data, base_data, stroke_sets, f_names):
    g, l = gl
    x, y = getGeneStrokePercentages(g, l)
    print(x)
    print(len(x))
    print(sum(x))
    print(y)
    print(len(y[0]))
    print(sum(y[0]))
    print()

In [10]:
# Test getGeneStrokePercentages() for accuracy
han_char = "4EFB" # 40, 46, 18, 6, 46, 6
# 40: variation of P more closely matches S2 (21, 0.2), not 40 - 0.03 match
# 46: 0.9 match, good
# 18: more closely matches H1 (6, 0.5), than H2 apparently - 0.01 match
# 6: split between 6 and 48 (D0???), 0.5
# some error results from format of archetypal strokes (ex. H1 is vertically centered at 0.5 instead of 0)
g, l, scale = loadRef(han_char)
x, y = getGeneStrokePercentages(g, l) 

#"""
for stroke in g:
    print(stroke)
    print()

print()
print(x)
print(len(x))
print(sum(x))
print(y)
print(len(y[0]))
print(sum(y[0]))
print()
#"""

6
_____
T1
[0.         0.75476272]
[1.         0.24523728]
[array([0.        , 0.75476272]), array([1.        , 0.24523728])]
[[140.      460.     ]
 [125.78125 428.78125]
 [110.625   398.625  ]
 [ 94.53125 369.53125]
 [ 77.5     341.5    ]
 [ 59.53125 314.53125]
 [ 40.625   288.625  ]
 [ 20.78125 263.78125]
 [  0.      240.     ]]

T1
points
[array([  0.       , 293.9522016]), array([140.       , 406.0477984])]
frac_prog
[0. 1.]
277.42474320137023
SWZ
[0 1]
[1 1]
[array([0, 1]), array([1, 1])]
[1 1]
[1 0]
[array([1, 1]), array([1, 0])]
[[140.      460.     ]
 [125.78125 428.78125]
 [110.625   398.625  ]
 [ 94.53125 369.53125]
 [ 77.5     341.5    ]
 [ 59.53125 314.53125]
 [ 40.625   288.625  ]
 [ 20.78125 263.78125]
 [  0.      240.     ]]

SWZ
points
[array([  0, 240]), array([140, 240]), array([140, 240]), array([140, 460])]
frac_prog
[0.         0.38888889 0.38888889 1.        ]
353.84406276663015
HZG1
[0. 0.]
[1. 0.]
[array([0., 0.]), array([1., 0.])]
[1. 0.]
[1. 1.]
[array([1., 0

[0. 0.]
[0.3 0. ]
[0.3 0. ]
[array([0., 0.]), array([0.3, 0. ])]
[0.3 0. ]
[0.319 0.799]
[1. 1.]
[array([0.3, 0. ]), array([0.319, 0.799]), array([1., 1.])]
[[ 90. 360.]
 [ 90.   0.]]

X
TN2
points
[array([ 90., 360.]), array([198., 360.]), array([198., 360.]), array([204.84,  72.36]), array([450.,   0.])]
frac_prog
[0.         0.16581277 0.16581277 0.6075523  1.        ]
559.3414490902974
SZZ2
[0.3 0. ]
[0.  0.5]
[0.  0.5]
[0.  0.5]
[array([0.3, 0. ]), array([0. , 0.5])]
[0.  0.5]
[1.  0.5]
[1.  0.5]
[1.  0.5]
[array([0. , 0.5]), array([1. , 0.5])]
[1.  0.5]
[0.79  0.783]
[0.344 0.976]
[0. 1.]
[array([1. , 0.5]), array([0.79 , 0.783]), array([0.344, 0.976]), array([0., 1.])]
[[ 90. 360.]
 [ 90.   0.]]

X
SZZ2
points
[array([198., 360.]), array([ 90., 180.]), array([ 90., 180.]), array([450., 180.]), array([450., 180.]), array([374.4 ,  78.12]), array([213.84,   8.64]), array([90.,  0.])]
frac_prog
[0.         0.21078492 0.21078492 0.57227808 0.57227808 0.69966993
 0.87534408 1.       

H1
points
[array([180., 410.]), array([420., 410.])]
frac_prog
[0. 1.]
127.49880575183148
N
[0. 0.]
[0.3 0.8]
[1. 1.]
[array([0., 0.]), array([0.3, 0.8]), array([1., 1.])]
[[420.     430.    ]
 [396.5625 420.625 ]
 [371.25   412.5   ]
 [344.0625 405.625 ]
 [315.     400.    ]
 [284.0625 395.625 ]
 [251.25   392.5   ]
 [216.5625 390.625 ]
 [180.     390.    ]]

N
points
[array([180., 430.]), array([252., 398.]), array([420., 390.])]
frac_prog
[0.         0.31901559 1.        ]
239.045267635446
HZT1
[0. 0.]
[0.5 0. ]
[array([0., 0.]), array([0.5, 0. ])]
[0.5 0. ]
[0.5 1. ]
[array([0.5, 0. ]), array([0.5, 1. ])]
[0.5 1. ]
[1.  0.5]
[array([0.5, 1. ]), array([1. , 0.5])]
[[420.     430.    ]
 [396.5625 420.625 ]
 [371.25   412.5   ]
 [344.0625 405.625 ]
 [315.     400.    ]
 [284.0625 395.625 ]
 [251.25   392.5   ]
 [216.5625 390.625 ]
 [180.     390.    ]]

HZT1
points
[array([180., 430.]), array([300., 430.]), array([300., 430.]), array([300., 390.]), array([300., 390.]), array([420., 41

[0.278 0.021]
[0.152 0.348]
[0.068 0.52 ]
[0.   0.67]
[array([0.278, 0.021]), array([0.152, 0.348]), array([0.068, 0.52 ]), array([0.  , 0.67])]
[0.   0.67]
[0.156 0.693]
[0.418 0.785]
[0.678 0.956]
[array([0.  , 0.67]), array([0.156, 0.693]), array([0.418, 0.785]), array([0.678, 0.956])]
[[420.     430.    ]
 [396.5625 420.625 ]
 [371.25   412.5   ]
 [344.0625 405.625 ]
 [315.     400.    ]
 [284.0625 395.625 ]
 [251.25   392.5   ]
 [216.5625 390.625 ]
 [180.     390.    ]]

PD2
points
[array([246.72, 429.16]), array([216.48, 416.08]), array([196.32, 409.2 ]), array([180. , 403.2]), array([180. , 403.2]), array([217.44, 402.28]), array([280.32, 398.6 ]), array([342.72, 391.76])]
frac_prog
[0.         0.14029214 0.23099537 0.30503413 0.30503413 0.46450324
 0.73270686 1.        ]
414.4196081953413
WG2
[0. 0.]
[0.304 0.172]
[0.456 1.   ]
[0.125 1.   ]
[array([0., 0.]), array([0.304, 0.172]), array([0.456, 1.   ]), array([0.125, 1.   ])]
[0.125 1.   ]
[0. 1.]
[0. 1.]
[0. 1.]
[array([0.125

[array([206., 550.]), array([140., 385.]), array([140., 385.]), array([470., 385.]), array([470., 385.]), array([450.2 , 225.28]), array([426.44, 247.39]), array([311.6, 269.5])]
frac_prog
[0.         0.21723451 0.21723451 0.62062887 0.62062887 0.81736625
 0.85704066 1.        ]
1051.0234036107918
D0
[0.  0.5]
[1.  0.5]
[array([0. , 0.5]), array([1. , 0.5])]
[[140. 220.]
 [470. 220.]]

D0
points
[array([140., 220.]), array([470., 220.])]
frac_prog
[0. 1.]
0.0
SG
[0.5 0. ]
[0.5 1. ]
[array([0.5, 0. ]), array([0.5, 1. ])]
[0.5 1. ]
[0.3 1. ]
[array([0.5, 1. ]), array([0.3, 1. ])]
[[140. 220.]
 [470. 220.]]

Y
SG
points
[array([305., 550.]), array([305., 220.]), array([305., 220.]), array([239., 220.])]
frac_prog
[0.         0.83333333 0.83333333 1.        ]
732.6201504914573
SZWG1
[0. 0.]
[0.  0.5]
[0.  0.5]
[0.  0.5]
[array([0., 0.]), array([0. , 0.5])]
[0.  0.5]
[1.  0.5]
[1.  0.5]
[1.  0.5]
[array([0. , 0.5]), array([1. , 0.5])]
[1.  0.5]
[0.94  0.984]
[0.868 0.917]
[0.52 0.85]
[array

[array([300., 210.]), array([312.92, 213.8 ]), array([347.88, 274.22]), array([386.64, 388.98]), array([300., 210.]), array([312.92, 206.2 ]), array([347.88, 145.78]), array([386.64,  31.02])]
frac_prog
[0.         0.02216281 0.13704022 0.33637996 0.66362004 0.68578284
 0.80066026 1.        ]
819.8726785713573
S1
[0.5 1. ]
[0.5 0. ]
[array([0.5, 1. ]), array([0.5, 0. ])]
[[300. 400.]
 [300.  20.]]

S1
points
[array([300.,  20.]), array([300., 400.])]
frac_prog
[0. 1.]
0.0
SZWG2
[0.2 0. ]
[0.  0.5]
[0.  0.5]
[0.  0.5]
[array([0.2, 0. ]), array([0. , 0.5])]
[0.  0.5]
[1.  0.5]
[1.  0.5]
[1.  0.5]
[array([0. , 0.5]), array([1. , 0.5])]
[1.  0.5]
[0.94  0.984]
[0.868 0.917]
[0.52 0.85]
[array([1. , 0.5]), array([0.94 , 0.984]), array([0.868, 0.917]), array([0.52, 0.85])]
[[300. 400.]
 [300.  20.]]

X
SZWG2
points
[array([376., 400.]), array([300., 210.]), array([300., 210.]), array([680., 210.]), array([680., 210.]), array([657.2 ,  26.08]), array([629.84,  51.54]), array([497.6,  77. ])]


[-0.01980198  1.01371585]
[0.0990099  1.01025137]
[1. 1.]
[array([0., 0.]), array([0.02970297, 0.91074681]), array([-0.03920792,  0.96903461]), array([-0.01980198,  1.01371585]), array([0.0990099 , 1.01025137]), array([1., 1.])]
[[170.  20.]
 [430.  20.]]

Y
C4
points
[array([170., 280.]), array([177.7227722,  43.2058294]), array([159.8059408,  28.0510014]), array([164.8514852,  16.433879 ]), array([195.742574 ,  17.3346438]), array([430.,  20.])]
frac_prog
[0.         0.44018452 0.48378419 0.50731601 0.56473436 1.        ]
560.1044491864991
PD1
[0.  0.5]
[0.034 0.49 ]
[0.126 0.331]
[0.228 0.029]
[array([0. , 0.5]), array([0.034, 0.49 ]), array([0.126, 0.331]), array([0.228, 0.029])]
[0.  0.5]
[0.034 0.51 ]
[0.126 0.669]
[0.228 0.971]
[array([0. , 0.5]), array([0.034, 0.51 ]), array([0.126, 0.669]), array([0.228, 0.971])]
[[170.  20.]
 [430.  20.]]

Y
PD1
points
[array([170., 150.]), array([178.84, 152.6 ]), array([202.76, 193.94]), array([229.28, 272.46]), array([170., 150.]), array([